In [1]:
#Import the right libraries 
import pandas as pd 
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pathlib
from distutils.dir_util import copy_tree
import os
import random
import shutil
import array as arr

In [6]:
import torch
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets, models
import tensorflow as tf

##Getting images just for train for now. Will get the other in later. But use the train first to test some type of model to see baseline errors 
BATCH_SIZE =40
IMG_HEIGHT = 200
IMG_WIDTH = 200

sets_dir = '~/test_instance/Sets'

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
        transforms.ToTensor()
    ]),
    'val': transforms.Compose([
        transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
        transforms.ToTensor()
    ]),
    'test': transforms.Compose([
        transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
        transforms.ToTensor()
    ])
}
directories = {'train':'Training', 'val':'Dev', 'test': 'Test'}
image_datasets = {x: datasets.ImageFolder(os.path.join(sets_dir, directories[x]),
                                          data_transforms[x])
                  for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val','test']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Class Names: " + str(class_names))
print(dataset_sizes)

Class Names: ['COVID', 'Non-COVID']
{'train': 1041, 'val': 130, 'test': 130}


In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

model_ft = models.resnet18(pretrained=False)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
import time
import copy
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

In [3]:
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

model_dn121 = models.densenet121(pretrained=False)

model_dn121 = model_dn121.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_dn121 = optim.Adam(model_dn121.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_dn121, step_size=7, gamma=0.1)

In [11]:
!pip install pytorch-ignite

     |████████████████████████████████| 103 kB 5.6 MB/s eta 0:00:01


In [4]:

import ignite
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
import ignite.metrics as mets

def thresholded_output_transform(output):
    y_pred, y = output
    y_pred = torch.round(y_pred)
    return y_pred, y

trainer = create_supervised_trainer(model_dn121, optimizer_dn121, criterion, device = device)
evaluator = create_supervised_evaluator(model_dn121, 
                                        metrics={'accuracy': mets.Accuracy(),'nll': 
                                                 mets.Loss(criterion), 'precision': 
                                                 mets.Precision(output_transform=thresholded_output_transform,average=False), 
                                                 'recall':mets.Recall(output_transform=thresholded_output_transform,average=False)}, 
                                        device=device)


In [5]:
@trainer.on(Events.ITERATION_COMPLETED(every=100))
def log_training(trainer):
    batch_loss = trainer.state.output
    lr = optimizer.param_groups[0]['lr']
    e = trainer.state.epoch
    n = trainer.state.max_epochs
    i = trainer.state.iteration
    print("Epoch {}/{} : {} - batch loss: {}, lr: {}".format(e, n, i, batch_loss, lr))

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(dataloaders['train'])
    metrics = evaluator.state.metrics
    print("Training Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}, Precision: {}, Recall: {} "
          .format(trainer.state.epoch, metrics['accuracy'], metrics['nll'], metrics['precision'][0:1], metrics['recall'][0:1]))

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(dataloaders['val'])
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}, Precision: {}, Recall: {}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['nll'], metrics['precision'][0:1], metrics['recall'][0:1]))

trainer.run(dataloaders['train'], max_epochs=25)

Training Results - Epoch: 1  Avg accuracy: 0.84 Avg loss: 0.51, Precision: tensor([0.], dtype=torch.float64), Recall: tensor([0.], dtype=torch.float64) 
Validation Results - Epoch: 1  Avg accuracy: 0.50 Avg loss: 1.46, Precision: tensor([0.], dtype=torch.float64), Recall: tensor([0.], dtype=torch.float64)
Training Results - Epoch: 2  Avg accuracy: 0.85 Avg loss: 0.54, Precision: tensor([0.9333], dtype=torch.float64), Recall: tensor([0.0824], dtype=torch.float64) 
Validation Results - Epoch: 2  Avg accuracy: 0.55 Avg loss: 1.67, Precision: tensor([1.], dtype=torch.float64), Recall: tensor([0.0615], dtype=torch.float64)
Training Results - Epoch: 3  Avg accuracy: 0.87 Avg loss: 0.33, Precision: tensor([0.8913], dtype=torch.float64), Recall: tensor([0.2412], dtype=torch.float64) 
Validation Results - Epoch: 3  Avg accuracy: 0.62 Avg loss: 0.86, Precision: tensor([1.], dtype=torch.float64), Recall: tensor([0.1846], dtype=torch.float64)


Current run is terminating due to exception: name 'optimizer' is not defined.
Engine run is terminating due to exception: name 'optimizer' is not defined.


NameError: name 'optimizer' is not defined